In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import warnings
warnings.filterwarnings('ignore')


#The code was referent by THIEN1504 kaggle website https://www.kaggle.com/code/thien1504/m5-simple-fast-model

In [2]:
cal=pd.read_csv('./data/calendar.csv')
sales_train_evaluation=pd.read_csv('data/sales_train_evaluation.csv')
sell_prices=pd.read_csv('data/sell_prices.csv')


In [3]:
for d in range(1942,1970):    
    col='d_'+str(d)
    sales_train_evaluation[col]=0
    sales_train_evaluation[col]=sales_train_evaluation[col].astype(np.int16)


In [4]:
def downcase(df):  
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

In [5]:
##Downcase data

cal=downcase(cal)
sales_train_evaluation=downcase(sales_train_evaluation)
sell_prices=downcase(sell_prices)



In [6]:
catalog_all=pd.melt(sales_train_evaluation,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='sold')
catalog_all=pd.merge(catalog_all,cal,on='d',how='left')
catalog_all=pd.merge(catalog_all,sell_prices,on=['store_id','item_id','wm_yr_wk'],how='left')


In [7]:
del cal,sell_prices,sales_train_evaluation
gc.collect

<function gc.collect(generation=2)>

In [8]:
catalog_all['sell_price']=catalog_all['sell_price'].fillna(catalog_all.groupby('id')['sell_price'].transform('median'))


In [9]:
d_id=dict(zip(catalog_all['id'].cat.codes,catalog_all['id']))
catalog_all['id']=catalog_all['id'].cat.codes
catalog_all['item_id']=catalog_all['item_id'].cat.codes
catalog_all['dept_id']=catalog_all['dept_id'].cat.codes
catalog_all['cat_id']=catalog_all['cat_id'].cat.codes
catalog_all['store_id']=catalog_all['store_id'].cat.codes
catalog_all['state_id']=catalog_all['state_id'].cat.codes
catalog_all['event_name_1']=catalog_all['event_name_1'].cat.codes
catalog_all['event_type_1']=catalog_all['event_type_1'].cat.codes

In [10]:
catalog_all['d']=catalog_all['d'].str[2:].astype(np.int16)

catalog_all=catalog_all.drop(['date','wm_yr_wk','event_name_2','event_type_2','weekday'],axis=1)

In [11]:
catalog_all['state_mean_sold']=catalog_all.groupby('state_id')['sold'].transform("mean").astype(np.float16)
catalog_all['store_mean_sold']=catalog_all.groupby('store_id')['sold'].transform("mean").astype(np.float16)
catalog_all['cat_mean_sold']=catalog_all.groupby('cat_id')['sold'].transform("mean").astype(np.float16)
catalog_all['dept_mean_sold']=catalog_all.groupby('dept_id')['sold'].transform("mean").astype(np.float16)
catalog_all['state_cat_mean_sold']=catalog_all.groupby(['state_id','cat_id'])['sold'].transform("mean").astype(np.float16)
catalog_all['state_dept_mean_sold']=catalog_all.groupby(['state_id','dept_id'])['sold'].transform("mean").astype(np.float16)
catalog_all['store_cat_mean_sold']=catalog_all.groupby(['store_id','cat_id'])['sold'].transform("mean").astype(np.float16)
catalog_all['cat_dept_mean_sold']=catalog_all.groupby(['dept_id','cat_id'])['sold'].transform("mean").astype(np.float16)
catalog_all['item_sold']=catalog_all.groupby('item_id')['sold'].transform("mean").astype(np.float16)
catalog_all['item_state_sold']=catalog_all.groupby(['item_id','state_id'])['sold'].transform("mean").astype(np.float16)
catalog_all['item_store_sold']=catalog_all.groupby(['item_id','store_id'])['sold'].transform("mean").astype(np.float16)
catalog_all['item_store_wday_sold']=catalog_all.groupby(['item_id','store_id','wday'])['sold'].transform("mean").astype(np.float16)

In [12]:
#time lag sold
lags=[1,2,6,7,12,14,30]
for lag in lags:
    catalog_all['sold_lag_'+str(lag)]= catalog_all.groupby(['id','item_id','dept_id','cat_id','store_id','state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)

In [13]:
catalog_all=catalog_all[ catalog_all['d'] > np.max(lags)]

In [14]:
catalog_all.to_pickle('data.pkl')